## Deep learning demystified: HIV protease substrate prediction #2

In the last post we tried to predict peptides cleaved by the HIV protease or not. While training our first deep neural network we observed that it was not *learning* but rather *remembering* the training data.
Here we will try to fix it and introduce some more deep learning concepts.

First we will quickly regenerate the training and test data

In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score


# let's read the files we dumped before

X_train = np.loadtxt("X_train.csv", delimiter=",")
X_val = np.loadtxt("X_val.csv",  delimiter=",")
y_train = np.loadtxt("y_train.csv" , delimiter=",")
y_val = np.loadtxt("y_val.csv", delimiter=",")
X_test = np.loadtxt("X_test.csv" , delimiter=",")
y_test = np.loadtxt("y_test.csv" , delimiter=",")

The last model that we used was this one were we used 2 hidden layers with 100 units each. 

In [53]:
from numpy.random import seed
seed(1)

model = Sequential()
model.add(Dense(units = 100, input_dim=X.shape[1], activation = 'relu', name='hidden1'))
model.add(Dense(units = 100, activation = 'relu', name='hidden2'))
model.add(Dense(units = 1, activation = 'sigmoid', name='output'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics=['acc'], lr=0.001)
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden1 (Dense)              (None, 100)               900       
_________________________________________________________________
hidden2 (Dense)              (None, 100)               10100     
_________________________________________________________________
output (Dense)               (None, 1)                 101       
Total params: 11,101
Trainable params: 11,101
Non-trainable params: 0
_________________________________________________________________


While this model performed good on the training data it did not seem to learn anything on the test data as the accuracy was stable for every epoch (cycle of training).
Several strategies for dealing with this situations are possible and we will try some of them.
First we will fit the original model and then create as before a radarplot to illustrate the differences

In [54]:
def metrics(model, X_test, y_test, nm):
    _, test_acc = model.evaluate(X_test, y_test, verbose=0)
    yhat_probs = model.predict(X_test, verbose=0)
    yhat_classes = model.predict_classes(X_test, verbose=0)
    yhat_probs = yhat_probs[:, 0]
    yhat_classes = yhat_classes[:, 0]

    accuracy = accuracy_score(y_test, yhat_classes)
    precision = precision_score(y_test, yhat_classes)
    recall = recall_score(y_test, yhat_classes)
    f1 = f1_score(y_test, yhat_classes)
    auc = roc_auc_score(y_test, yhat_probs)
    return [nm, accuracy, precision, recall, f1, auc] 



history = model.fit(X_train, y_train, 
                    validation_data=(X_val, y_val), 
                    epochs=100, 
                    batch_size=256, 
                    # set verbose = 1 to have full output
                    verbose=0
                    )
base_perf = metrics(model, X_test, y_test, nm='base dnn')
print(base_perf)

['base dnn', 0.9019607843137255, 0.9072847682119205, 0.7287234042553191, 0.8082595870206489, 0.9239081746920493]


### Reduce model size!

While random forest or ensamble algorithms benefits by using the largest possible number of classifier neural networks are very prone to overfit in large networks and is important to understand the relationship between the number of features, the number of neurons and the number of layers.


So our architecture (2 layers with 100 neurons) might just be *too powerful for our task!*
Let's try a easier one


In [63]:
model = Sequential()
model.add(Dense(units = 50, input_dim=X.shape[1], activation = 'relu', name='hidden1'))
model.add(Dense(units = 50, input_dim=X.shape[1], activation = 'relu', name='hidden2'))
model.add(Dense(units = 1, activation = 'sigmoid', name='output'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics=['acc'], lr=0.001)
model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden1 (Dense)              (None, 50)                450       
_________________________________________________________________
hidden2 (Dense)              (None, 50)                2550      
_________________________________________________________________
output (Dense)               (None, 1)                 51        
Total params: 3,051
Trainable params: 3,051
Non-trainable params: 0
_________________________________________________________________


In [64]:
history8neuron = model.fit(X_train, y_train, 
                           validation_data=(X_val, y_val), 
                           epochs=100, 
                           batch_size=32, 
                           verbose=0
                           )
simple_dnn_perf = metrics(model, X_test, y_test, nm='dropout dnn')
simple_dnn_perf

['dropout dnn',
 0.8883861236802413,
 0.8131868131868132,
 0.7872340425531915,
 0.7999999999999999,
 0.9294064949608061]

### Batch size and learning


Batch size as we seen before is a even size chunk of data. For example we used 128 which means the model will go through the entire dataset 128 peptides at the time. 

- *Smaller batches are noisy (as they resemble less the original training data) and can help with overfitting* 
- Large batches represents better the training data and help the model converge to a *global optimum* rather than a local optimum

In [33]:

history32batch = model.fit(X_train, y_train, 
                           validation_data=(X_val, y_val), 
                           epochs=100, 
                           batch_size=32, 
                           verbose=0
                           )

batch32_perf = metrics(model, X_test, y_test, nm='32 batch size dnn')


history32batch = model.fit(X_train, y_train, 
                           validation_data=(X_val, y_val), 
                           epochs=100, 
                           batch_size=64, 
                           verbose=0
                           )

batch64_perf = metrics(model, X_test, y_test, nm='64 batch size dnn')

### Dropout to increase generalization

A popular way to increase the model capability to generalize is to use Dropout. The idea is that we can remove randomly some connection at every epoch (i.e *making the model forget things*), thereby forcing the model to not rely on specific connections.

In [35]:
from tensorflow.keras.layers import Dropout

model = Sequential()
model.add(Dense(units = 100, input_dim=X.shape[1], activation = 'tanh', name='hidden1'))
# drop 20% of connections
model.add(Dropout(0.2))
model.add(Dense(units = 100, activation = 'relu', name='hidden2'))
# drop 30 % of connection
model.add(Dropout(0.2))
model.add(Dense(units = 1, activation = 'sigmoid', name='output'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics=['acc'], lr=0.001)
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden1 (Dense)              (None, 100)               900       
_________________________________________________________________
hidden2 (Dense)              (None, 100)               10100     
_________________________________________________________________
output (Dense)               (None, 1)                 101       
Total params: 11,101
Trainable params: 11,101
Non-trainable params: 0
_________________________________________________________________


In [37]:
historydropout = model.fit(X_train, y_train, 
                           validation_data=(X_val, y_val), 
                           epochs=100, 
                           batch_size=32, 
                           verbose=0
                           )
dropout_perf = metrics(model, X_test, y_test, nm='dropout dnn')
dropout_perf

['dropout dnn',
 0.7646699266503667,
 0.1744186046511628,
 0.2073732718894009,
 0.18947368421052632,
 0.5461397816986714]